In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 6
height = 5
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 30

puzzle.naming = {
    None: ' ',
    'e': '_',
}

puzzle.domain_program += """
    value(e).
    num(2..4).
    value(N) :- num(N).
    1 { guessed_number(num_nums,4..15) } 1.
    max_id(N) :- guessed_number(num_nums,N).
    id(N) :- max_id(N).
    id(N-1) :- id(N), N > 1.
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : solution(C,V) }.
    :- puzzle(C,e).
"""
puzzle.puzzle_constraints_program += """
    #maximize { 1,solution(C,V) : solution(C,V), num(V), not puzzle(C,V) }.
    :- not solution(C,V) : cell(C), num(V). 
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""
puzzle.solution_program += enc_library['cell_distance']
puzzle.solution_program += enc_library['adjacent_cells']
puzzle.solution_program += """
    solution(C) :- solution(C,V), num(V).
    first_solution(C1) :- solution(C1), C1 <= C2 : solution(C2).

    1 { map(C,I) : id(I) } 1 :- solution(C).
    1 { map(C,I) : solution(C) } 1 :- id(I).
    :- not map(C,1), first_solution(C).

    next_id(I,I+1) :- id(I), id(I+1).
    next_id(I,1) :- max_id(I).
    :- map(C1,I), map(C2,J), next_id(I,J), solution(C1,D), not cell_distance(C1,C2,D).
    
    forbidden_pair(D,E) :- num(D), num(E), |D-E| > 1.
    :- map(C1,I), map(C2,J), next_id(I,J),
        solution(C1,D), solution(C2,E), forbidden_pair(D,E).
    
    :- guessed_number(num_nums,N),
        not N { solution(C,V) : solution(C,V), num(V) } N.
    
    :- adjacent_cells(C1,C2),
        solution(C1,D), num(D),
        solution(C2,E), num(E).
"""

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

Done grounding..
....
Stopped after solving timeout..
Solving time: 70.81 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

|3| |2| |4| |
| | | |3| | |
| | |2| | | |
|4| | | |4| |
| |4| | | | |

num_nums = 12


In [5]:
print(puzzle.pretty_repr_solution())

|3|_|2|_|4|_|
|_|4|_|3|_|4|
|_|_|2|_|_|_|
|4|_|_|_|4|_|
|_|4|_|3|_|3|
